In [1]:
import sys
import os

sys.path.append(os.path.abspath('..\..'))

import yaml
import pandas as pd
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.features.features import feature_engineering
from src.models.retrieval import candidate_generation
from src.features.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.evaluator import evaluation

In [2]:
# read config
with open('..\config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

In [5]:
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

In [6]:
candidates = candidate_generation(df_train, config['optimization']['retrieval'])
df_train = pd.concat([df_train.iloc[:,:3], candidates['positive'], candidates['negative']], ignore_index=True)

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features)
    for df in (df_train, df_valid)
    ]

del candidates, user_item_features

In [7]:
# perform bayesian search
searcher = BayesianSearch(config['optimization']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("Best trial:\n", study.best_trial.params)
del searcher

[I 2025-04-25 12:59:37,771] A new study created in memory with name: no-name-c28a2e03-6a5d-4c45-a9ca-271a655c48ea
[I 2025-04-25 12:59:43,314] Trial 0 finished with value: 0.9473724259819775 and parameters: {'learning_rate': 0.23249108771063184, 'gamma': 7.893363520730975, 'max_depth': 9, 'subsample': 0.8548384207408888, 'n_estimators': 131}. Best is trial 0 with value: 0.9473724259819775.
[I 2025-04-25 12:59:57,060] Trial 1 finished with value: 0.9471506901001964 and parameters: {'learning_rate': 0.01637393864471923, 'gamma': 7.5662374982049165, 'max_depth': 8, 'subsample': 0.7675852090064104, 'n_estimators': 346}. Best is trial 0 with value: 0.9473724259819775.
[I 2025-04-25 13:00:08,465] Trial 2 finished with value: 0.9478075371859467 and parameters: {'learning_rate': 0.13962990478284204, 'gamma': 8.330999579356412, 'max_depth': 6, 'subsample': 0.7318297041242805, 'n_estimators': 325}. Best is trial 2 with value: 0.9478075371859467.
[I 2025-04-25 13:00:16,670] Trial 3 finished with v

Best trial:
 {'learning_rate': 0.13962990478284204, 'gamma': 8.330999579356412, 'max_depth': 6, 'subsample': 0.7318297041242805, 'n_estimators': 325}


In [8]:
# use indicator to know the origin
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
# keep only rows that are present in df1 but not in df2
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative and positive sampling
candidates = candidate_generation(df_train, config['optimization']['retrieval'])
df_train_ = pd.concat([df_train.iloc[:,:3], candidates['positive'], candidates['negative']], ignore_index=True)

df_train_, df_test_ = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train_, df_test)
    ]

del candidates, user_item_features

In [9]:
from xgboost import XGBRanker

clf = XGBRanker(**study.best_trial.params)
clf.fit(
    df_train_['X'], df_train_['y'].astype(int)
    , group=df_train_['group']
    , verbose=False
    )

preds = clf.predict(df_test_['X'])

print(evaluation(df_test_['y'], preds, df_test_['group']))

0.9524184211563961


In [10]:
def calculate_metrics(top_items, data):
    # Coverage: proportion of unique items recommended
    recommended_items = top_items['item_id'].unique()
    total_items = data['item_id'].unique()
    coverage_score = len(recommended_items) / len(total_items)
    
    # Novelty: Based on item popularity (inverse)
    item_popularity = data.groupby('item_id').size()
    item_popularity = item_popularity[item_popularity > 0]
    item_popularity = 1 - item_popularity / item_popularity.sum()  # Normalize
    novelty_score = top_items['item_id'].map(item_popularity).mean()

    return coverage_score, novelty_score

# Calculate metrics for the recommendations
coverage_score, novelty_score = calculate_metrics(df_test.iloc[:, :2], df_train.iloc[:, :3])
# Print out the results
print(f'Coverage: {coverage_score}')
print(f'Novelty: {novelty_score}')

Coverage: 0.6463195691202872
Novelty: 0.9984584266355963


In [11]:
from src.models.baseline import baseline_model

result = baseline_model(dataframes=dfs, n=10)
group = result.groupby('user_id').size().to_list()

print(evaluation(result['rating'], result['est_rating'], group))
del group

0.9168812846621082


In [12]:
user_item_features = feature_engineering(dataframes={
    'user': dfs['user'], 'item': dfs['item'], 'data': dfs['data']
    })

df_baseline = build_rank_input(ratings=result.drop(columns=['est_rating']), features=user_item_features)

preds = clf.predict(df_baseline['X'])
print(evaluation(df_baseline['y'], preds, df_baseline['group']))

0.9478088569953991
